In [5]:
def shuffler(df):
  return df.reindex(np.random.permutation(df.index))

In [7]:
# Load all the data from the CSV file 
PT_DATA_PATH = "../../../Dataset/Botnet_Detection/PT_838_Security Camera"
XC_DATA_PATH = "../../../Dataset/Botnet_Detection/XCS7_1002_WHT_Security_Camera"
df_pt = pd.read_csv(PT_DATA_PATH+"/benign_traffic.csv")
df_xc = pd.read_csv(XC_DATA_PATH+"/benign_traffic.csv")

In [8]:
df_pt = df_pt.assign(label = 'pt')
df_xc = df_xc.assign(label = 'xc')

In [9]:
df_all = df_pt
df_all = df_all.append(df_xc, ignore_index = True)

In [10]:
df_all = shuffler(df_all)

In [11]:
#Divide dataframe into x and y
df_x = pd.DataFrame(df_all.iloc[:, 0:115])
df_y = pd.DataFrame(df_all.iloc[:, 115])

In [14]:
#Normalize the x dataframe
df_x = normalize(df_x)

In [15]:
#One-Hot encoding labels and transform into array
df_y = label_encoder(df_y)

[array(['pt', 'xc'], dtype=object)]


In [16]:
df_y

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [17]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(df_x, df_y, test_size=0.20)

In [20]:
class Net(torch.nn.Module):
    def __init__(self, in_dim, h_dim, out_dim):
        super(Net, self).__init__()
        self.linear1 = torch.nn.Linear(in_dim, h_dim)
        self.bn1 = nn.BatchNorm1d(h_dim)
        self.linear2 = torch.nn.Linear(h_dim, out_dim)
        
    def forward(self, x):
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred

In [18]:
train_x = torch.FloatTensor(train_x.values.astype(np.float32))
test_x = torch.tensor(test_x.values.astype(np.float32))
train_y = torch.tensor(train_y.astype(np.float32))
test_y = torch.tensor(test_y.astype(np.float32))

In [42]:
train_x.shape

torch.Size([116079, 115])

In [43]:
test_x.shape

torch.Size([29020, 115])

In [44]:
test_y.shape

torch.Size([29020, 2])

In [51]:
test_x.mean(), test_x.std()

(tensor(0.1194), tensor(0.1794))

In [19]:
epochs = 600
input_dim = 115
output_dim = 2 #Number of clasees
h_dim = 100
lr_rate = 1e-6

In [21]:
model = Net(input_dim, h_dim, output_dim)
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)

In [28]:
n_test, y_bm = test_y.shape

In [47]:
def training(epochs, model, train_data, labels, vali_data, target):
    for e in range(int(epochs)):
        y_pred = model(train_data)

        # Compute and print loss
        loss = criterion(y_pred, labels)
        if e % 100 == 99:
            loss_f = float(loss)
            print(e, "Loss:", loss_f)
            total = n_test
            correct = 0.0
            outputs = model(vali_data)
            _b, pred = torch.max(outputs, 1)
            vb, label = torch.max(target, 1)
            correct+= float((pred == label).sum())
            accuracy = float(100*(correct/total))
            print('Accuracy: {:.4f}'.format(accuracy))

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [48]:
training(epochs, model, train_x, train_y, test_x, test_y) ## Train the initial model on Server

99 Loss: 1730.869873046875
BM Accuracy: 99.5693
199 Loss: 1681.443115234375
BM Accuracy: 99.5865
299 Loss: 1632.770751953125
BM Accuracy: 99.5968
399 Loss: 1592.5736083984375
BM Accuracy: 99.6313
499 Loss: 1539.232177734375
BM Accuracy: 99.6347
599 Loss: 1490.8829345703125
BM Accuracy: 99.6416


In [ ]:
def computer():
    

In [ ]:
def OMP(y, V, sl=10, ep=0.001):
    support_set = []
    k = 1
    while k<= sl or np.linalg.norm(y) <= ep:
        _, i = torch.max()
        